In [1]:
import os
import torch
from torch import nn,optim
from torch.utils.data import DataLoader,Dataset
from torchvision import datasets
import torchvision.models as models
from torchvision.transforms import ToTensor, Lambda, Compose,transforms
import matplotlib.pyplot as plt
import pandas as pd
from torchvision.io import read_image
import pandas as pd
import numpy as np
import cv2 as cv
from PIL import Image
from matplotlib import cm
from torch.nn.functional import interpolate

In [2]:
dataset_location = 'C:/Users/sushl/Desktop/deep learning by stanford/synthetic image feature detection/synthetic-image-feature-detection-using-deep-learning/dataset/New folder'
L = os.listdir(dataset_location)
L.sort()
img_path = []
for i in range(int(len(L))):
    img_path.append(dataset_location +"/"+ L[i])
dataframe = {"path":img_path}
df = pd.DataFrame(dataframe)
df.to_csv("test.csv")

In [3]:
test = pd.read_csv("test.csv")
test

,Unnamed: 0,path
0,0,C:/Users/sushl/Desktop/deep learning by stanfo...
1,1,C:/Users/sushl/Desktop/deep learning by stanfo...
2,2,C:/Users/sushl/Desktop/deep learning by stanfo...
3,3,C:/Users/sushl/Desktop/deep learning by stanfo...
4,4,C:/Users/sushl/Desktop/deep learning by stanfo...
...,...,...
233,233,C:/Users/sushl/Desktop/deep learning by stanfo...
234,234,C:/Users/sushl/Desktop/deep learning by stanfo...
235,235,C:/Users/sushl/Desktop/deep learning by stanfo...
236,236,C:/Users/sushl/Desktop/deep learning by stanfo...


In [4]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file,transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = self.img_labels.iloc[idx, 1]
        image = read_image(img_path)
        img = cv.imread(img_path,0)
        image = image/255
        #print(image.dtype)
        gray = np.float32(img)
        dst = cv.cornerHarris(gray,2,3,0.04)
        dst = dst/(dst.max()-dst.min()+0.00001)
        x1 = np.where(dst > 0.05)[0]
        y1 = np.where(dst > 0.05)[1]
        n = len(x1)
        #print(n)
        label12 = np.stack((x1, y1), axis = 1)
        blur = cv.getGaussianKernel(19,4)
        blur = (blur + blur.T)/2
        blur = blur/(blur.max()-blur.min())
        label_img = np.zeros(img.shape)
        for k in range(len(label12)):
            label_img[label12[k,1]-10:label12[k,1]+9,label12[k,0]-10:label12[k,0]+9] = blur*255
        #label = Image.fromarray(np.uint8(cm.gist_earth(label_img)*255))
        label_img = cv.GaussianBlur(label_img,(11,11),0)
        label = torch.from_numpy(label_img.T.reshape(1,200,200))
        label = label/255
        #print(label_img.)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        sample = (image,label)
        return sample

In [5]:
test_data = CustomImageDataset(
    annotations_file= 'test.csv',
    transform=transforms.Compose([transforms.Resize((128,128))]),
    target_transform=transforms.Compose([transforms.Resize((128,128))])
)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)
test_features, test_labels = next(iter(test_dataloader))

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [11]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, kernel_size=1, stride=1, padding=0)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=1, stride=1, padding=0)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=1, stride=1, padding=0)
        self.conv4 = nn.Conv2d(32, 64, kernel_size=1, stride=1, padding=0)
        self.convT1 = nn.Conv2d(64, 32, kernel_size=1, stride=1, padding=0)
        self.convT2 = nn.Conv2d(32, 16, kernel_size=1, stride=1, padding=0)
        self.convT3 = nn.Conv2d(16, 8, kernel_size=1, stride=1, padding=0)
        self.convT4 = nn.Conv2d(8, 1, kernel_size=1, stride=1, padding=0)
        self.relu = nn.LeakyReLU()
        self.mp = nn.MaxPool2d(2, 2)
        self.mup = nn.MaxUnpool2d(2,2)
        self.sigmoid = nn.Sigmoid()
        self.batchnorm1 = nn.BatchNorm2d(8)
        self.batchnorm2 = nn.BatchNorm2d(16)
        self.batchnorm3 = nn.BatchNorm2d(32)
        self.batchnorm4 = nn.BatchNorm2d(64)
        
        self.encoder1 = nn.Sequential(
            nn.Conv2d(8,8,kernel_size = 3,stride=1,padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(8,8,kernel_size = 3,stride=1,padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(8,8,kernel_size = 3,stride=1,padding=1),
        )
        self.encoder2 = nn.Sequential(
            nn.Conv2d(16,16,kernel_size = 3,stride=1,padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(16,16,kernel_size = 3,stride=1,padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(16,16,kernel_size = 3,stride=1,padding=1),
        )
        self.encoder3 = nn.Sequential(
            nn.Conv2d(32,32,kernel_size = 3,stride=1,padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(32,32,kernel_size = 3,stride=1,padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(32,32,kernel_size = 3,stride=1,padding=1),
        )
        self.encoder4 = nn.Sequential(
            nn.Conv2d(64,64,kernel_size = 3,stride=1,padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(64,64,kernel_size = 3,stride=1,padding=1),
            nn.LeakyReLU(),
            nn.Conv2d(64,64,kernel_size = 3,stride=1,padding=1),
        )
        self.decoder1=nn.Sequential(
            nn.ConvTranspose2d(64,64,kernel_size = 3,stride=1,padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(64,64,kernel_size = 3,stride=1,padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(64,64,kernel_size = 3,stride=1,padding=1),
        )
        self.decoder2=nn.Sequential(
            nn.ConvTranspose2d(32,32,kernel_size = 3,stride=1,padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(32,32,kernel_size = 3,stride=1,padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(32,32,kernel_size = 3,stride=1,padding=1),
        )
        self.decoder3=nn.Sequential(
            nn.ConvTranspose2d(16,16,kernel_size = 3,stride=1,padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(16,16,kernel_size = 3,stride=1,padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(16,16,kernel_size = 3,stride=1,padding=1),
        )
        self.decoder4=nn.Sequential(
            nn.ConvTranspose2d(8,8,kernel_size = 3,stride=1,padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(8,8,kernel_size = 3,stride=1,padding=1),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(8,8,kernel_size = 3,stride=1,padding=1),
        )
    def forward(self, features):
        x = self.encode(features.float())
        x = self.decode(x)
        return x
    
    def encode(self, features):
        x = self.conv1(features.float()) #3 to 8
        residual = x
        x = self.encoder1(x)
        x = 0.5*(x + residual)
        x = self.relu(x)
        x = self.mp(x)
        x = self.batchnorm1(x)

        x = self.conv2(x) #8 to 16
        residual = x
        x = self.encoder2(x)
        x = 0.5*(x + residual)
        x = self.relu(x)
        x = self.batchnorm2(x)
        
        residual = x #16 to 16
        x = self.encoder2(x)
        x = 0.5*(x + residual)
        x = self.relu(x)
        x = self.batchnorm2(x)
        
        residual = x #16 to 16
        x = self.encoder2(x)
        x = 0.5*(x + residual)
        x = self.relu(x)
        x = self.mp(x)
        x = self.batchnorm2(x)
        
        x = self.conv3(x) #16 to 32
        residual = x
        x = self.encoder3(x)
        x = 0.5*(x + residual)
        x = self.relu(x)
        x = self.batchnorm3(x)
        
        residual = x #32 to 32
        x = self.encoder3(x)
        x = 0.5*(x + residual)
        x = self.relu(x)
        x = self.batchnorm3(x)
        
        residual = x #32 to 32
        x = self.encoder3(x)
        x = 0.5*(x + residual)
        x = self.relu(x)
        x = self.mp(x)
        x = self.batchnorm3(x)
        
        x = self.conv4(x) #32 to 64
        residual = x
        x = self.encoder4(x)
        x = 0.5*(x + residual)
        x = self.relu(x)
        x = self.batchnorm4(x)
        
        residual = x #64 to 64
        x = self.encoder4(x)
        x = 0.5*(x + residual)
        x = self.relu(x)
        x = self.batchnorm4(x)
        
        residual = x #64 to 64
        x = self.encoder4(x)
        x = 0.5*(x + residual)
        x = self.relu(x)
        x = self.batchnorm4(x)
        
        return x

    def decode(self, features):
        #x = interpolate(features.float(), scale_factor=2)
        residual = features.float() #64 to 64
        x = self.decoder1(features.float())
        x = 0.5*(x + residual)
        x = self.relu(x)
        x = self.batchnorm4(x)
        
        residual = x #64 to 64
        x = self.decoder1(x)
        x = 0.5*(x + residual)
        x = self.relu(x)
        x = self.batchnorm4(x)
         
        x = self.convT1(x) #64 to 32
        x = interpolate(x, scale_factor=2)
        residual = x
        x = self.decoder2(x)
        x = 0.5*(x + residual)
        x = self.relu(x)
        x = self.batchnorm3(x)
        
        residual = x #32 to 32
        x = self.decoder2(x)
        x = 0.5*(x + residual)
        x = self.relu(x)
        x = self.batchnorm3(x)
        
        residual = x #32 to 32
        x = self.decoder2(x)
        x = 0.5*(x + residual)
        x = self.relu(x)
        x = self.batchnorm3(x)
        
        x = self.convT2(x) #32 to 16
        x = interpolate(x, scale_factor=2)
        residual = x 
        x = self.decoder3(x)
        x = 0.5*(x + residual)
        x = self.relu(x)
        x = self.batchnorm2(x)
        
        residual = x #16 to 16
        x = self.decoder3(x)
        x = 0.5*(x + residual)
        x = self.relu(x)
        x = self.batchnorm2(x)
        
        residual = x  #16 to 16
        x = self.decoder3(x)
        x = 0.5*(x + residual)
        x = self.relu(x)
        x = self.batchnorm2(x)
        
        x = self.convT3(x) #16 to 8
        x = interpolate(x, scale_factor=2)
        residual = x 
        x = self.decoder4(x)
        x = 0.5*(x + residual)
        x = self.relu(x)
        x = self.batchnorm1(x)
        
        residual = x #8 to 8
        x = self.decoder4(x)
        x = 0.5*(x + residual)
        x = self.relu(x)
        x = self.batchnorm1(x)
        
        residual = x #8 to 8
        x = self.decoder4(x)
        x = 0.5*(x + residual)
        x = self.relu(x)
        x = self.batchnorm1(x)
        
        x = self.convT4(x) #8 to 1
        x = self.sigmoid(x)
        return x
    
#model = CNN()#.to(device)
checkpoint = torch.load('C:/Users/sushl/Desktop/deep learning by stanford/synthetic image feature detection/synthetic-image-feature-detection-using-deep-learning/features.pth')
model.load_state_dict(checkpoint)

AttributeError: 'CNN' object has no attribute 'copy'